In [184]:
import pandas as pd
import operator
import itertools

In [185]:
def gcd(a, b):
    r_0, r_1 = a, b 
    u_0, u_1 = 1, 0
    v_0, v_1 = 0, 1

    while r_1 != 0:
        q = r_0 // r_1
        r_0, r_1 = r_1, r_0 - q*r_1
        u_0, u_1 = u_1, u_0 - q*u_1
        v_0, v_1 = v_1, v_0 - q*v_1

    return r_0, u_0, v_0

def inv(a, b):
    d, u, v = gcd(a, b)
    if d != 1:
        return "No inverse exists"
    else:
        return u

def mod_comp(a, b, n):
    d = gcd(a, n)[0]
    s = []

    if d == 1:
        return (inv(a, n) * b) % n, 1
    elif b % d != 0: 
        return "No solutions.", 0
    else:
        a = a / d
        b = b / d
        n = n / d
        x_0 = (inv(a, n) * b) % n

        for i in range(0, d):
            s.append(x_0 + i * n)

        return s, d


In [186]:
alp = ['а','б','в','г','д','е','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ы','ь','э','ю','я']
m = len(alp)

f = open('V7 (1).txt', 'r', encoding='utf-8')
text = f.read()
f.close()

def bigrams_without_sp(text):
    count = {}
    t = 1
    while t < len(text):
        p = text[t - 1]
        l = text[t]
        if (p + l) not in count:
            count[p + l] = 1
        if (p + l) in count:  
            count[p + l] = count[p + l] + 1
        t = t + 2
    return count


a = sorted(bigrams_without_sp(text).items(), key=lambda x:x[1], reverse = True)

fr_bigrams_txt = []
for i in range(0, 5):
    fr_bigrams_txt.append(a[i][0])

fr_bigrams_lng = ['ст', 'но', 'то', 'на', 'ен']


In [187]:
def match(y):
    return alp.index(y[0]) * m + alp.index(y[1])

def unmatch(X):
    x1 = X // m
    x2 = X - x1*m
    return x1, x2

def get_keys(a, b):
    keys = []
    for i in range(len(a)):
        for k in range(len(b)):
            for j in range(len(a)):
                    for l in range(len(b)):
                        if i!=j:
                            if k!=l:
                                x1 = match(a[i])
                                x2 = match(a[j])
                                y1 = match(b[k])
                                y2 = match(b[l])
                                y = (y1 - y2) % (m **2)
                                x = (x1 - x2) % (m **2)
                                a1, t = mod_comp(y, x, m **2)
                                if t == 1:
                                    b1 = (y - a1 * x) % (m **2)
                                elif t == 0:
                                    continue
                                else:
                                    b1 = []
                                    b1.append((y - i * x) % (m **2) for i in a1)
                                keys.append((a1, b1))
    return keys

In [188]:
def decypher(k, text):
    text_n = ""
    Y = list(bigrams_without_sp(text).keys())
    X = {}
    for i in range(0, len(Y)):
        X[i] = inv(k[0], m**2) * (match(Y[i]) - k[1]) 
        X[i] = X[i] % (m **2)

    t = 1
    while t < len(text):
        p = text[t - 1]
        l = text[t]
        ind = Y.index(p + l)
        x1, x2 = unmatch(X[ind])
        text_n += alp[x1] + alp[x2]
        t = t + 2

    return text_n

print(decypher(k, text))


TypeError: 'int' object is not subscriptable